<img src="https://docs.dask.org/en/latest/_images/dask_horizontal.svg" align="right" width="30%"/>

<center>
    <h1>
Dask Tutorial for PyHEP 2022
    </h1>
</center>

Dask is a pure-Python, parallel, and distributed computing library designed to scale up workflows in the PyData ecosystem.

Dask is has two main components:

- The **collections library(ies)** (sometimes called "Dask core")
  - `dask.array`: chunked NumPy
  - `dask.dataframe`: partitioned Pandas
  - `dask.bag`: partitioned iterables
  - `dask.delayed`: custom algorithms
- The **execution engines** (task schedulers)
  - The distributed engine is its own project (`distributed`, sometimes called "Dask Distributed")

<div style="text-align: center;">
  <img src="https://docs.dask.org/en/stable/_images/dask-overview.svg" align="center" width="70%"/>
</div>